In [1]:
from grand import lig_samplers as ls
from grand import lig_utils as lu

import openmm
import openmm.unit
import openff.toolkit
import pdbfixer

import numpy as np
import mdtraj as md
from scipy.spatial import Voronoi, Delaunay
import os

import Bio.PDB

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
datadir = '../../examples/fcrn/'

In [3]:
ligands = {
    "ETH":"CC",
    "EOL":"CCO",
    "THS":"CC(C)O",
    "BUT":"CC(C)(C)O",
    "ACN":"CC#N",
    "AMN":"CN",
    "DFO":"CN(C)C=O",
    "DME":"COC",
    "BDY":"C1=CC=C(C=C1)C=O ",
    "BEN":"C1=CC=CC=C1",
    "CHX":"C1CCCCC1",
    "PHN":"C1=CC=C(C=C1)O",
    "ACD":"CC(=O)N",
    "ACT":"CC(=O)C",
    "ADY":"CC=O",
    "URE":"C(=O)(N)N",
}

In [4]:
lig_ff_dir = f'{datadir}/lig_ff/'

ref_coords = []
ref_dict = {}
pdb_parser = Bio.PDB.PDBParser(QUIET=True)
struct = pdb_parser.get_structure('ml_pocket',file=f'{datadir}/pocket_miner.pdb')
atoms = struct.get_atoms()
for atom in atoms:
    if atom.name == 'CA':
        if atom.bfactor > 0.7:
            ref_dict[atom.parent.id[1]] = atom.coord/10

In [5]:
ref_dict

{11: array([-0.703    , -2.3207002, 15.083501 ], dtype=float32),
 55: array([-1.0459, -2.5128, 17.175 ], dtype=float32),
 60: array([-1.0949   , -3.2417998, 16.7392   ], dtype=float32),
 61: array([-0.7652, -3.0947, 16.6112], dtype=float32),
 64: array([-0.9002, -3.153 , 16.1299], dtype=float32),
 67: array([-0.8862001, -3.4008   , 15.674899 ], dtype=float32),
 71: array([-0.7188  , -3.1926  , 15.082899], dtype=float32),
 94: array([-1.5005  , -2.4292  , 15.126101], dtype=float32),
 120: array([-1.8399   , -2.8379998, 14.3533   ], dtype=float32),
 206: array([-1.5238, -0.4982, 16.2116], dtype=float32),
 208: array([-1.8235,  0.0335, 16.1862], dtype=float32),
 226: array([-0.97910005, -0.058     , 15.6515    ], dtype=float32),
 228: array([-0.6675  , -0.5611  , 16.076601], dtype=float32),
 236: array([-1.0298,  0.2164, 15.99  ], dtype=float32)}

In [6]:
# Define reference CA atoms (as residue id) per pocket
pocket_dict = {
    'p1':[206,208,226,228,236],
}

In [7]:
for pocket_key in pocket_dict.keys():
    ref_coords = []
    for p_key in pocket_dict[pocket_key]:
        ref_coords.append(ref_dict[p_key])
    print(ref_coords)
    if not os.path.exists(f'{datadir}/{pocket_key}'):
        os.mkdir(f'{datadir}/{pocket_key}')
    for lig_key in ligands.keys():
        savedir = f'{datadir}/{pocket_key}/{lig_key}/'
        if not os.path.exists(savedir):
            os.mkdir(savedir)
        # Load protein and ligand
        prot = openmm.app.PDBFile(f'{datadir}/input.pdb')
        lig = openff.toolkit.Molecule.from_smiles(ligands[lig_key])
        lig.generate_conformers(n_conformers=1000)
        lig_top = lig.to_topology().to_openmm()
        lig_pos = lig.conformers[0].to_openmm()

        # Load forcefield
        ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml',f'{lig_ff_dir}/{lig_key}_openff.xml')

        # Add one ligand to the system
        sys_top, sys_pos, ghosts = lu.add_ghosts(prot.topology, prot.positions, lig_top, lig_pos, n=1, output=f'{datadir}/{savedir}/ncmc_{lig_key}_init.pdb')

        # Load pdb with protein and ligand
        pdb = openmm.app.PDBFile(f'{datadir}/{savedir}/ncmc_{lig_key}_init.pdb')

        frag_res_ids = []
        frag_atom_ids = []
        alchemical_atom_ids = []
        for resid, residue in enumerate(pdb.topology.residues()):
            if residue.name == "UNK":
                frag_res_ids.append(resid)
                for atom in residue.atoms():
                    frag_atom_ids.append(atom.index)
                    alchemical_atom_ids.append(atom.index)

        pdb_system = ff.createSystem(pdb.topology, nonbondedMethod=openmm.app.NoCutoff)
        dcd_reporter = openmm.app.DCDReporter(f"{datadir}/{savedir}/ncmc_{lig_key}_fcrn.dcd", reportInterval=1000000)
        state_reporter = openmm.app.StateDataReporter(f"{datadir}/{savedir}/ncmc_{lig_key}_fcrn_state.csv",step=True, time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, reportInterval=1000000)
        
        # Initialize Sampler
        sampler = ls.BaseNCMCSampler(pdb_system,pdb.topology,pdb.positions,reporters=[dcd_reporter,state_reporter],alchemical_atoms=alchemical_atom_ids, overwrite=True,
                                     nsteps_neq=1000,nsteps_eq=1000, frag_atoms=frag_atom_ids, insert_points_list=ref_coords
                                    )  
        
        mean, std = sampler.compute_mfpt(100,5)
        print(lig_key, sampler.fpt_list, mean, std)

        with open(f'{datadir}/output_{pocket_key}.txt','a') as f:
            f.write(f'{lig_key}\t{sampler.fpt_list}\t{mean}\t{std}')

[array([-1.5238, -0.4982, 16.2116], dtype=float32), array([-1.8235,  0.0335, 16.1862], dtype=float32), array([-0.97910005, -0.058     , 15.6515    ], dtype=float32), array([-0.6675  , -0.5611  , 16.076601], dtype=float32), array([-1.0298,  0.2164, 15.99  ], dtype=float32)]


  0%|          | 0/5 [00:00<?, ?it/s]

-3.819122314453125 kJ/mol 348.0644226074219 kJ/mol 850.4567108154297 kJ/mol 1063.7509765625 kJ/mol rejected


100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


-3.824798583984375 kJ/mol 292.8992004394531 kJ/mol 846.0630340576172 kJ/mol 752.064453125 kJ/mol rejected


...

100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


59.30537414550781 kJ/mol 17.290206909179688 kJ/mol 533.4713439941406 kJ/mol 746.2421875 kJ/mol rejected


100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


67.78227233886719 kJ/mol 151.53274536132812 kJ/mol 655.3750457763672 kJ/mol 987.3212890625 kJ/mol rejected


100%|██████████| 1/1 [00:02<00:00,  2.32s/it]
